In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

In [3]:
# sys.path.append("..")

In [4]:
import nlp_tools
from cycontext.viz import visualize_ent, visualize_dep

In [5]:
text = """
History of Present Illness:
the patient presents today with diabetes and r/o for pneumonia.
"""

In [24]:
nlp = nlp_tools.build_nlp(model="en_core_web_sm")

In [25]:
nlp.pipe_names

['tagger',
 'parser',
 'ner',
 'target_matcher',
 'context',
 'sectionizer',
 'postprocessor']

In [26]:
doc = nlp(text)

In [27]:
visualize_ent(doc)

# I. Preprocessor

In [28]:
preprocessor = nlp.tokenizer

In [29]:
preprocessor

In [12]:
import re

In [30]:
preprocess_rules = [
    lambda x: x.lower(),
    nlp_tools.PreprocessingRule(re.compile("pt"), "patient"),
#     nlp_tools.PreprocessingRule(re.compile("r/o"), "rule out"),
]

In [31]:
preprocessor.add(preprocess_rules)

In [32]:
nlp(text)


history of present illness:
the patient presents today with diabetes and r/o for pneumonia.

# II. TargetMatcher

In [33]:
target_matcher = nlp.get_pipe("target_matcher")

In [34]:
target_matcher

In [35]:
target_rules = [
    nlp_tools.TargetRule("diabetes", "PROBLEM"),
    nlp_tools.TargetRule("pneumonia", "PROBLEM")
]

In [36]:
target_matcher.add(target_rules)

In [37]:
doc = nlp(text)

In [38]:
doc.ents

(today, diabetes, pneumonia)

In [39]:
visualize_ent(doc)

# III. ConText

In [40]:
from cycontext.viz import visualize_dep, visualize_ent

In [41]:
visualize_ent(doc)

In [42]:
visualize_dep(doc)

In [43]:
for ent in doc.ents:
    print(ent._.is_uncertain)

False
False
True


# IV. Postprocessor

In [44]:
postprocessor = nlp.get_pipe("postprocessor")

In [45]:
from nlp_tools import postprocessing_functions

In [46]:
postprocess_rules = [
    nlp_tools.PostprocessingRule(
        patterns=[
            nlp_tools.PostprocessingPattern(lambda x: x._.is_uncertain),
        ],
        action=postprocessing_functions.remove_ent
    ),
    
]

In [47]:
postprocessor.add(postprocess_rules)

In [48]:
doc = nlp(text)

In [49]:
doc.ents

(today, diabetes)

In [50]:
visualize_ent(doc)